In [59]:
with open("rosalind_ba2d.txt", "r") as f:
        k, t = map(int, f.readline().split())
        DNA = [line.strip() for line in f]
print(k)
print(t)
print(DNA)

12
25
['CGGTCTACCGCCACGAAGCTTCCGACGATGTCTGATAGCGAGTGGGCAGGCGGCGTTCGCTTATACCGCGTAGCATAGTATTATCATCGGAGAGGATGTTTGTCAAACGTGAGTGGGCATCATTAGCACCCTCACGTTCAGACAAAGCGAACGAAT', 'TTCGAAAGCTCCGCACCCCATTTCCTTAATCGGGCAACCAAGTCAGAATTCGGGTGTCATAGTGAGGGGGCACCCTAACTATAAAAGAGTCCATCTGCTACTAAGGGGCCAAGCGGCGACAGTGGCAAGGTGTCTGGGATGCTGTTCTTATCATAG', 'CTGCATGCGATAGTCCCGAGTCACGGGGTAGGTCGTGAAACGAAGCCGATTGAGAGGGCTCGTCATCCGATTACCGGAGATTGGGGTATTTGAGCACCTGGTTGATGGGCATCTACTTGGGTTAACTGAGCCAATACGCTATTCGGATTGGGATGT', 'TCATATAAAAGGAGCAGCGTCTATTATTTCTGAATGGATGAATCAAGACCGAAGGATTTAGCACTTGCGCTTTCCTTCTCTTGAGGCTAGCGACACTGTGTTTGATGGACTTAAGCGGCGGTCGTGTGGCGCAAAGCTCCGTCAAGAGAGCGGGCC', 'GTCGGGTAGTTGGAATGAACCGGCGATTAATAGCGCGCATGTTCTTAAACCAGTCCAATGGTGCCGCGAATGACTCATATTAGGATAGAGAGGGCCATAGCGAGGAAATTAAGCAATTTTTGAGAACCGCGCAAGGTGTTCTGATGACGCCTCGGG', 'AAGCAACGGTAAACTGAGAGGGCGAGTCGCCGAAGTTAAGTTGCGCGGGTAAGTGCAAGGGGGTGCGTGTGTTGATCGCCATTTGTGCACTCATGTGCACTCTGCACGGTCCGACACCTGGACTTATGTGCGTTCTTTGTGTCTCATGCTTTCTAT', 'TTTCCAAGCTATTTTTTAGCAATAGAAGATTA

In [60]:
def HammingDistance(string1, string2):
    
    "This function calculates the Hamming Distance between two strings of equal length."
    
    # check if strings are the same length
    # alternate: assert len(string1) == len(string2), "Strings must be same length!"
    if len(string1) != len(string2):
        print("Strings must be the same length!")
        
    number_mismatches = 0
    string_length = len(string1)
    for i in range(string_length):
        if string1[i] != string2[i]:
            number_mismatches += 1

    return number_mismatches

In [61]:
def ProfileMostProbableList(Dna, k, Profile):
    kmer_probability_max = 0
    max_kmer = ""
    pos = {'A' : 0, 'C' : 1, 'G' : 2, 'T' : 3}
    for i in range(len(Dna) - k + 1):
        kmer = Dna[i : i + k]
        # kmer_probabilty must be set to 1 because it will be multiplied by subsequent probs
        kmer_probability = 1
        # iterate throuh kmers and multiple probabilities from Profile dictionaries
        for j, basepair in enumerate(kmer):
            # index through dictionary for value i in key basepair
            probability = Profile[pos[basepair]][j]
            # multiple probability by existing probability_value
            kmer_probability *= probability
        
        # check if max_kmer is ""
        if not max_kmer:
            max_kmer = kmer
            kmer_probability_max = kmer_probability
        # check if kmer_probability is largest value seen so far, replace variable if so
        elif kmer_probability > kmer_probability_max:
            kmer_probability_max = kmer_probability
            max_kmer = kmer
    return max_kmer

In [62]:
def Profile(passed_motifs, k):
    "Creates a profile matrix for each passed profile matrix."
    # create empty matrix of floats
    matrix = []
    for i in range(4):
        # make k number of [0.0] entries in matrix for each spot in kmer
        matrix.append([0.0] * k)
        
    # for each position in kmer, count bases
    number_motifs = len(passed_motifs)
    for i in range(k):
        motif_count = {"A" : 0, "C" : 0, "G" : 0, "T" : 0}
        for motif in passed_motifs:
            motif_count["A"] += motif[i].count("A")
            motif_count["C"] += motif[i].count("C")
            motif_count["G"] += motif[i].count("G")
            motif_count["T"] += motif[i].count("T")

        # create matrix of profiles for each base
        matrix[0][i] = motif_count["A"] / number_motifs
        matrix[1][i] = motif_count["C"] / number_motifs
        matrix[2][i] = motif_count["G"] / number_motifs
        matrix[3][i] = motif_count["T"] / number_motifs
        
    return matrix

In [63]:
Profile(['GGC', 'AAG'],3)

[[0.5, 0.5, 0.0], [0.0, 0.0, 0.5], [0.5, 0.5, 0.5], [0.0, 0.0, 0.0]]

In [64]:
def Score(passed_motifs):
    "This function scores differences between passed motifs and a consensus kmer from probability profiles."
    k = len(passed_motifs[0])
    consensus = []
    for i in range(k):
        freq = {"A" : 0, "C" : 0, "G" : 0, "T" : 0}
        for motif in passed_motifs:
            freq["A"] += motif[i].count("A")
            freq["C"] += motif[i].count("C")
            freq["G"] += motif[i].count("G")
            freq["T"] += motif[i].count("T")
    
        # based on freq above, creat a consensus kmer to compare to passed motif
        max_freq = max(freq.values())
        for nt, count in freq.items():
            if count == max_freq:
                consensus.append(nt)
                break
                
    consensus = "".join(consensus)
    score_value = 0
    for motif in passed_motifs:
        score_value += HammingDistance(motif, consensus)
    
    return score_value

In [65]:
def GreedyMotifSearch(k, t, Dna):
    # create list of best motifs from out of the first DNA string given
    best_motifs = []
    for seq in Dna:
        first = seq[0:k]
        best_motifs.append(first)
    # best_motifs = [seq[:k] for seq in dna]
    
    # iterate over kmers in first Dna string, create a motif list for each kmer
    first_seq = Dna[0]
    for start in range(len(first_seq) - k + 1):
        kmer = first_seq[start : start + k]
        # start motif list based on this kmer from first_seq
        motif = [kmer]
        
        # iterate over subsequent Dna strings, make  profile from them based on first_seq kmer
        for i in range(1, t):
            matrix = Profile(motif, k)
            most_probable = ProfileMostProbableList(Dna[i], k, matrix)
            motif.append(most_probable)
            
        # score motif, replace if best
        if Score(motif) < Score(best_motifs):
            best_motifs = motif
            
    return best_motifs

In [66]:
x = GreedyMotifSearch(k,t,DNA)
for i in x:
    print(i)

CGTGAGTGGGCA
TTCGAAAGCTCC
CTGCATGCGATA
TCATATAAAAGG
TTGGAATGAACC
AAGCAACGGTAA
TAGTAAACATCA
TTGTATCGAGCG
ACCGATTGGTCA
CGCGATAAAACA
TCGTAGCGGACA
AGGGAGCGGGCG
ACGGAGCGGGCA
ACGGAGGGGGCG
AAGTAGCGCTTA
ATCGAGAGGGCA
ATTGAGAGGGCC
AGCGAGCGGGCC
ATGGAGTGGGCC
CTGCAACGCACG
TTCTAGCGCTCG
ATAGAGAGGGCA
AGAGAGTGGGCC
CTTGATTGAGGA
AGCGAGTGGGCC
